In [2]:
import os
import pandas as pd
import shutil
import tempfile
import matplotlib.pyplot as plt
import numpy as np
import PIL
import torch
from torch import nn
import torchvision.models
import cv2

from torchvision.transforms import Lambda, Normalize
from torchvision import transforms
from PIL import Image

from torchvision import models
from torchvision.models import resnet18
from monai.transforms import (
    Activations,
    AddChannel,
    Compose,
    LoadPNG,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity, Transpose, 
    LoadImage,
    ToTensor,
    Resize,
)
from monai.utils import set_determinism
set_determinism(seed=0)

act = Activations(sigmoid=True)

In [3]:
class Race_CNN(nn.Module): 
    def __init__(self):
        
        super(Race_CNN, self).__init__()
        self.resnet18 = resnet18(pretrained=True)
        num_ftrs = self.resnet18.fc.in_features
        self.resnet18.fc = torch.nn.Linear(num_ftrs, 1)

    def forward(self, x):
        output = self.resnet18(x)
        output = torch.squeeze(output, dim = -1)
        return output

In [4]:
cuda_device = "cuda:0"
model = Race_CNN().cuda()
device = torch.device(cuda_device)
model.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 44.3MB/s]


Race_CNN(
  (resnet18): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [5]:
model.load_state_dict(torch.load("../praveer/race_detection/models/best_metric_model.pth"))

<All keys matched successfully>

In [7]:
import os
ls = os.listdir("pytorch-CycleGAN-and-pix2pix-master/results/rop_seg_data_gfrozen5_lrby2/test_35_setB_black/images/")

In [8]:
len(ls)
ls.sort()

In [9]:
data = []
for i in range(len(ls)//2):
    data.append([ls[2*i], "WHITE"])
    data.append([ls[2*i+1], "BLACK"])

In [10]:
combined = pd.DataFrame(data, columns=["image_id", "race"])

In [11]:
dict_label2idx = {'BLACK':int(0),'WHITE':int(1)}

In [12]:
df_test = combined.replace({'race': dict_label2idx})

In [13]:
df_test = df_test.rename(columns={"image_id": "imageName", "race": "label"})

In [14]:
class RaceDataset(torch.utils.data.Dataset):
    def __init__(self, image_files, labels, transforms):
        self.image_files = image_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)
        
    def __getitem__(self, index):
        return self.transforms(self.image_files[index]), self.labels[index], self.image_files[index]

In [15]:
test_paths_old = [str(f) for f in df_test['imageName']]
test_image_class = df_test['label'].tolist()

In [16]:
test_image_class_list = []
test_image_files_list_updated = []

for i, elem in enumerate(test_paths_old):
    test_image_files_list_updated.append("pytorch-CycleGAN-and-pix2pix-master/results/rop_seg_data_gfrozen5_lrby2/test_35_setB_black/images/"+elem)
    test_image_class_list.append(test_image_class[i])

In [18]:
test_transforms = Compose(
    [
        LoadPNG(image_only=True),
        ToTensor(),
    ]
)

In [19]:
test_ds = RaceDataset(test_image_files_list_updated, test_image_class_list, test_transforms)

In [20]:
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=16, num_workers=4)

In [21]:
model.eval()
with torch.no_grad():
    y_pred = torch.tensor([], dtype=torch.float32, device=device)
    y = torch.tensor([], dtype=torch.long, device=device)
    test_filenames_all = []
    for test_data in test_loader:
        test_images, test_labels = (
            test_data[0].to(device),
            test_data[1].to(device),
        )
        test_images = test_images.permute((0, 3, 2, 1))
        
        test_filenames = test_data[2]
        test_filenames_all.extend(test_filenames)

        y_pred = torch.cat([y_pred, model(test_images)], dim=0)
        y = torch.cat([y, test_labels], dim=0)
        print(y_pred)
    df_out = pd.DataFrame({'filename':test_filenames_all, 'predictions': act(y_pred).cpu().numpy()})
    filename = os.path.join('Race_classification_test35.csv')
    df_out.to_csv(filename,index=False)

tensor([ -94.1333,  -36.2949,  -69.8825,  -11.2481,  -67.0406,   -6.9888,
        -127.3418,   -4.3725,  -88.1186,  -17.7929,  -93.3150,  -20.0687,
         -59.8951,  -18.7934,  -98.4149,  -36.8931], device='cuda:0')
tensor([ -94.1333,  -36.2949,  -69.8825,  -11.2481,  -67.0406,   -6.9888,
        -127.3418,   -4.3725,  -88.1186,  -17.7929,  -93.3150,  -20.0687,
         -59.8951,  -18.7934,  -98.4149,  -36.8931,  -41.5332,  -47.5352,
         -71.0326,  -90.5635, -125.3887,  -24.5664,  -48.9716,  -16.3186,
         -36.4559,   -1.9806,  -95.7484,  -84.4849,  -67.4986,  -40.3524,
         -62.5843,  -44.7459], device='cuda:0')
tensor([ -94.1333,  -36.2949,  -69.8825,  -11.2481,  -67.0406,   -6.9888,
        -127.3418,   -4.3725,  -88.1186,  -17.7929,  -93.3150,  -20.0687,
         -59.8951,  -18.7934,  -98.4149,  -36.8931,  -41.5332,  -47.5352,
         -71.0326,  -90.5635, -125.3887,  -24.5664,  -48.9716,  -16.3186,
         -36.4559,   -1.9806,  -95.7484,  -84.4849,  -67.4986,  -40.

tensor([ -94.1333,  -36.2949,  -69.8825,  -11.2481,  -67.0406,   -6.9888,
        -127.3418,   -4.3725,  -88.1186,  -17.7929,  -93.3150,  -20.0687,
         -59.8951,  -18.7934,  -98.4149,  -36.8931,  -41.5332,  -47.5352,
         -71.0326,  -90.5635, -125.3887,  -24.5664,  -48.9716,  -16.3186,
         -36.4559,   -1.9806,  -95.7484,  -84.4849,  -67.4986,  -40.3524,
         -62.5843,  -44.7459,  -27.7289,   -2.1890,  -87.7135,  -34.8680,
        -131.2592, -113.9665, -135.3274, -101.8671,  -97.5959,  -77.9131,
         -59.4293,  -63.3483, -135.6798,  -26.3589,    2.7629,   21.5266,
         -79.1547,  -45.4461,  -13.4472,   -7.5041,  -14.5156,   12.5075,
         -56.0978,    2.0358,  -20.4857,  -14.2314,  -57.8005,  -41.1117,
          -9.3546,   -2.6575,  -71.0363,  -89.8441, -131.2718, -113.9696,
         -77.0080,  -21.7722,  -71.8620,  -16.3426,  -63.3886,  -24.8569,
         -56.3211,  -41.5403, -197.9600,  -69.5933,  -10.8382,   -7.3049,
         -28.9594,  -26.4342,   -8.718

tensor([ -94.1333,  -36.2949,  -69.8825,  -11.2481,  -67.0406,   -6.9888,
        -127.3418,   -4.3725,  -88.1186,  -17.7929,  -93.3150,  -20.0687,
         -59.8951,  -18.7934,  -98.4149,  -36.8931,  -41.5332,  -47.5352,
         -71.0326,  -90.5635, -125.3887,  -24.5664,  -48.9716,  -16.3186,
         -36.4559,   -1.9806,  -95.7484,  -84.4849,  -67.4986,  -40.3524,
         -62.5843,  -44.7459,  -27.7289,   -2.1890,  -87.7135,  -34.8680,
        -131.2592, -113.9665, -135.3274, -101.8671,  -97.5959,  -77.9131,
         -59.4293,  -63.3483, -135.6798,  -26.3589,    2.7629,   21.5266,
         -79.1547,  -45.4461,  -13.4472,   -7.5041,  -14.5156,   12.5075,
         -56.0978,    2.0358,  -20.4857,  -14.2314,  -57.8005,  -41.1117,
          -9.3546,   -2.6575,  -71.0363,  -89.8441, -131.2718, -113.9696,
         -77.0080,  -21.7722,  -71.8620,  -16.3426,  -63.3886,  -24.8569,
         -56.3211,  -41.5403, -197.9600,  -69.5933,  -10.8382,   -7.3049,
         -28.9594,  -26.4342,   -8.718

In [22]:
a = df_out.predictions

In [24]:
b = [a[2*i] for i in range(358//2) if a[2*i] > 0.5]